In [1]:
import pandas as pd
import nltk
import re

In [33]:
import glob
import io
import numpy as np

## Job Titles list

In [2]:
with open('CityofLA/Additional data/job_titles.csv', encoding = 'utf-8') as f:
    print(f.readlines())

['311 DIRECTOR\n', 'ACCOUNTANT\n', 'ACCOUNTING CLERK\n', 'ACCOUNTING RECORDS SUPERVISOR\n', 'ADMINISTRATIVE ANALYST\n', 'ADMINISTRATIVE CLERK\n', 'ADMINISTRATIVE HEARING EXAMINER\n', 'ADVANCE PRACTICE PROVIDER CORRECTIONAL CARE\n', 'AIR CONDITIONING MECHANIC\n', 'AIR CONDITIONING MECHANIC SUPERVISOR\n', 'AIRPORT AIDE\n', 'AIRPORT CHIEF INFORMATION SECURITY OFFICER\n', 'AIRPORT ENGINEER\n', 'AIRPORT GUIDE\n', 'AIRPORT INFORMATION SPECIALIST\n', 'AIRPORT MANAGER\n', 'AIRPORT POLICE CAPTAIN\n', 'AIRPORT POLICE LIEUTENANT\n', 'AIRPORT POLICE OFFICER\n', 'AIRPORT POLICE SPECIALIST\n', 'AIRPORT SUPERINTENDENT OF OPERATIONS\n', 'AIRPORTS MAINTENANCE SUPERINTENDENT\n', 'AIRPORTS MAINTENANCE SUPERVISOR\n', 'AIRPORTS PUBLIC AND COMMUNITY RELATIONS DIRECTOR\n', 'ANIMAL CARE ASSISTANT\n', 'ANIMAL CARE TECHNICIAN\n', 'ANIMAL CARE TECHNICIAN SUPERVISOR\n', 'ANIMAL CONTROL OFFICER\n', 'ANIMAL KEEPER\n', 'APPARATUS OPERATOR\n', 'APPLICATIONS PROGRAMMER\n', 'APPRENTICE - METAL TRADES\n', 'APPRENTICE MA

In [3]:
with open('CityofLA/Additional data/kaggle_data_dictionary.csv', encoding = 'utf-8') as f:
    print(f.readlines())

['Field Name,Annotation Letter,Description,Data Type,Allowable Values,Accepts Null Values?,Additional Notes\n', 'FILE_NAME,,The file name of the job bulletin from which each record came,String,,No,\n', 'JOB_CLASS_TITLE,A,"The title of the particular class (e.g., Systems Analyst, Carpenter)",String,Any job title matching in in supplied job bulletins,No,\n', 'JOB_CLASS_NO,B,The 4-digit code that corresponds to the job class.,String,Any four character numeric string matching one in supplied job bulletins,No,"Some classes have three non-zero digits — e.g., 0162. These will take the form 0###"\n', 'REQUIREMENT_SET_ID,D1,"A main requirement that has to be filled. These are identified with numbers in the job bulletin. Relationships between requirement sets are specified by an ""AND|OR"" conjunction (data element E1 on the data annotation chart)",Integer,Any positive integer,No,\n', 'REQUIREMENT_SUBSET_ID,D2,"A secondary requirement that has to be filled, nested under a REQUIREMENT_SET_ID. The

In [4]:
job_titles = pd.read_csv('CityofLA/Additional data/job_titles.csv')

## Data Dictionary

In [5]:
kaggle_data_dictionary = pd.read_csv('CityofLA/Additional data/kaggle_data_dictionary.csv')

In [6]:
kaggle_data_dictionary

,Field Name,Annotation Letter,Description,Data Type,Allowable Values,Accepts Null Values?,Additional Notes
0,FILE_NAME,NaN,The file name of the job bulletin from which e...,String,NaN,No,NaN
1,JOB_CLASS_TITLE,A,"The title of the particular class (e.g., Syste...",String,Any job title matching in in supplied job bull...,No,NaN
2,JOB_CLASS_NO,B,The 4-digit code that corresponds to the job c...,String,Any four character numeric string matching one...,No,Some classes have three non-zero digits — e.g....
3,REQUIREMENT_SET_ID,D1,A main requirement that has to be filled. Thes...,Integer,Any positive integer,No,NaN
4,REQUIREMENT_SUBSET_ID,D2,"A secondary requirement that has to be filled,...",String,Any letter or combination of letters.,No,"A-Z represents 1-26. In the case of 27th, 28th..."
5,JOB_DUTIES,C,A summary of what someone does in the particul...,String,NaN,No,NaN
6,EDUCATION_YEARS,F,Years required in a particular education program,Float,Any positive float representing years or a fra...,Yes,Any job description that mentions months shoul...
7,SCHOOL_TYPE,G,School Type: School type required (e.g. colleg...,String,"COLLEGE OR UNIVERSITY, HIGH SCHOOL, APPRENTICE...",Yes,Allowable value list is most likely incomplete...
8,EDUCATION_MAJOR,H,Major or concentration required,String,NaN,Yes,A list of many sufficient majors should appear...
9,EXPERIENCE_LENGTH,I,Years required in a particular job class or ex...,Float,NaN,Yes,Some classes may specify months rather than ye...


In [7]:
kaggle_data_dictionary.iloc[3]['Description']

'A main requirement that has to be filled. These are identified with numbers in the job bulletin. Relationships between requirement sets are specified by an "AND|OR" conjunction (data element E1 on the data annotation chart)'

## Sample 

In [8]:
sample = pd.read_csv('CityofLA/Additional data/sample job class export template.csv')

In [9]:
sample.head()

,FILE_NAME,JOB_CLASS_TITLE,JOB_CLASS_NO,REQUIREMENT_SET_ID,REQUIREMENT_SUBSET_ID,JOB_DUTIES,EDUCATION_YEARS,SCHOOL_TYPE,EDUCATION_MAJOR,EXPERIENCE_LENGTH,...,COURSE_LENGTH,COURSE_SUBJECT,MISC_COURSE_DETAILS,DRIVERS_LICENSE_REQ,DRIV_LIC_TYPE,ADDTL_LIC,EXAM_TYPE,ENTRY_SALARY_GEN,ENTRY_SALARY_DWP,OPEN_DATE
0,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,1,A,"A Systems Analyst analyzes procedures, methods...",4.0,COLLEGE OR UNIVERSITY,COMPUTER SCIENCE|INFORMATION SYSTEMS|GEOGRAPHI...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17
1,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,2,A,"A Systems Analyst analyzes procedures, methods...",4.0,COLLEGE OR UNIVERSITY,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17
2,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,2,B,"A Systems Analyst analyzes procedures, methods...",4.0,COLLEGE OR UNIVERSITY,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17
3,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,2,C,"A Systems Analyst analyzes procedures, methods...",4.0,COLLEGE OR UNIVERSITY,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17
4,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,3,A,"A Systems Analyst analyzes procedures, methods...",NaN,NaN,NaN,2.0,...,"3M, 4Q",INFORMATION SYSTEMS|SYSTEMS ANALYSIS|closely r...,At least three of the courses must be from the...,M,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17


In [10]:
text = ''
with open('CityofLA/Job Bulletins/ACCOUNTING RECORDS SUPERVISOR 1119 072718.txt', encoding = 'utf-8') as f:
    text = f.read()

In [19]:
job_title = text.splitlines()[0].strip()

In [12]:
pattern = r'Class Code:\s*(\d*)'
class_code = re.findall(pattern, text)[0]

In [13]:
pattern = r'Open Date:\s*(\d\d-\d\d-\d\d)'
open_date = re.findall(pattern, text)[0]

In [14]:
pattern = r'(\$\d+[,]*\d+\D+\$\d+[,]*\d+)'
salary_range = re.findall(pattern, text)

In [15]:
pattern = r'DUTIES\W+(.*\n)'
duties = re.findall(pattern, text)

In [16]:
pattern = r'REQUIREMENT(S)?(/MINIMUM\sQUALIFICATION)?\W+(.*\n)'
requirement = re.findall(pattern, text)[0][2]

In [17]:
pattern = r'(MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY)\W+(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)(.*)'
x = re.findall(pattern, text)[0]
deadline = ''.join(x).strip()

In [18]:
code = r'Class Code:\s*(\d*)'
open_d = r'Open Date:\s*(\d\d-\d\d-\d\d)'
sal = r'(\$\d+[,]*\d+\D+\$\d+[,]*\d+)'
dut = r'DUTIES\W+(.*\n)'
req = r'REQUIREMENT(S)?(/MINIMUM\sQUALIFICATION)?\W+(.*\n)'
end_d = r'(MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY)\W+(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)(.*)'

In [20]:
def doextraction(glob_text):
    df = pd.DataFrame() 
    """Get all the files from the given glob and pass them to the extractor."""
    for thefile in glob.glob(glob_text)[:200]:
        with io.open(thefile, 'r', encoding='utf-8') as fyl:
            text = fyl.read()
            df = get_features(text, thefile, df)
            
    return df

In [36]:
def get_features(text, filename, df):
    
    job_title = text.splitlines()[0].strip()
    class_code = re.findall(code, text)[0]
    try:
        open_date = re.findall(open_d, text)[0]
    except:
        open_date = np.NaN
    salary_range = re.findall(sal, text)
    duties = re.findall(dut, text)
    requirement = re.findall(req, text)[0][2]
    try:
        x = re.findall(end_d, text)[0]
        deadline = ''.join(x).strip()
    except:
        deadline = np.NaN
    df = df.append({'FILE_NAME': filename, 'JOB_CLASS_TITLE': job_title, 'JOB_CLASS_NO': class_code, 'SALARY_RANGE': salary_range,
                   'DUTIES': duties, 'REQUIREMENTS': requirement, 'APPLICATION_DEADLINE': deadline}, ignore_index= True)
    
    return df

In [37]:
data_df = doextraction('CityofLA/Job Bulletins/*.txt')

In [41]:
data_df

,APPLICATION_DEADLINE,DUTIES,FILE_NAME,JOB_CLASS_NO,JOB_CLASS_TITLE,REQUIREMENTS,SALARY_RANGE
0,"THURSDAYMAY 1, 2014.",[A 311 Director is responsible for the success...,CityofLA/Job Bulletins\311 DIRECTOR 9206 0418...,9206,311 DIRECTOR,1. One year of full-time paid experience as a ...,"[$125,175 to $155,514]"
1,"SATURDAYAUGUST 25, 2018, in Los Angeles. Candi...",[An Accountant does professional accounting wo...,CityofLA/Job Bulletins\ACCOUNTANT 1513 062218.txt,1513,ACCOUNTANT,Graduation from an accredited four-year colleg...,"[$49,903 to $72,996, $55,019 to $80,472]"
2,NaN,[An Accounting Clerk performs difficult and re...,CityofLA/Job Bulletins\ACCOUNTING CLERK 1223 0...,1223,ACCOUNTING CLERK,Two years of full-time paid office clerical ex...,"[$49,005 to $71,618]"
3,"THURSDAYAUGUST 9, 2018.","[An Accounting Records Supervisor assigns, rev...",CityofLA/Job Bulletins\ACCOUNTING RECORDS SUPE...,1119,ACCOUNTING RECORDS SUPERVISOR,Two years of full-time paid experience as an A...,"[$55,332 to $80,930, $65,145 to $95,254]"
4,"THURSDAYJUNE 14, 2018.",[An Administrative Analyst performs profession...,CityofLA/Job Bulletins\ADMINISTRATIVE ANALYST ...,1590,ADMINISTRATIVE ANALYST,MINIMUM QUALIFICATIONS\n,"[$60,489 to $88,468, $71,451 to $104,462]"
5,"THURSDAYAPRIL 12, 2018.",[An Administrative Clerk performs general offi...,CityofLA/Job Bulletins\ADMINISTRATIVE CLERK 13...,1358,ADMINISTRATIVE CLERK,Six months of full-time paid experience in cle...,"[$37,584 to $54,935, $43,785 (flat rated); $43..."
6,"THURSDAYOCTOBER 22, 2015.",[An Administrative Hearing Examiner conducts q...,CityofLA/Job Bulletins\ADMINISTRATIVE HEARING ...,9135,ADMINISTRATIVE HEARING EXAMINER,1. Two years of full-time paid experience with...,"[$54,726 to $80,012]"
7,NaN,[An Advance Practice Provider Correctional Car...,CityofLA/Job Bulletins\ADVANCE PRACTICE PROVID...,2325,ADVANCE PRACTICE PROVIDER CORRECTIONAL CARE,1. Must be licensed as a: \n,"[$78,487 to $97,530, $84,459 to $104,922]"
8,"THURSDAYAPRIL 27, 2017.","[An Air Conditioning Mechanic installs, modifi...",CityofLA/Job Bulletins\AIR CONDITIONING MECHAN...,3774,AIR CONDITIONING MECHANIC,MINIMUM QUALIFICATIONS\n,"[$90,118 (flat-rated)\nThe salary in the Depar..."
9,"THURSDAYNOVEMBER 29, 2018.",[An Air Conditioning Mechanic Supervisor assig...,CityofLA/Job Bulletins\AIR CONDITIONING MECHAN...,3781,AIR CONDITIONING MECHANIC SUPERVISOR,Four years of full-time paid experience as an ...,"[$103,841 and $108,482, $127,388 to $134,488]"
